In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from PIL import Image

In [17]:
class CLIP_Similarity(nn.Module):
    def __init__(self, concept_vector, img_dir):
        super(CLIP_Similarity, self).__init__()
        random_number = random.randint(1, 2000)
        random.seed(random_number)
        np.random.seed(random_number)
        torch.manual_seed(random_number)
        cudnn.benchmark = False
        cudnn.deterministic = True
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model, self.preprocess = clip.load("/home/ubuntu/Downloads/ViT-L-14.pt", device=device)
        raw_image = Image.open(img_dir).convert("RGB")
        self.image = self.preprocess(raw_image).unsqueeze(0).to(device)
        self.concept_vector = concept_vector
        
    
    def forward(self, image):
        image_features = self.model.encode_image(image)
        sim = torch.cosine_similarity(image_features, self.concept_vector, dim=1)
        sim = torch.sum(sim)
        return sim 
        
        
    
    

# Concept of violence

In [12]:
import clip
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("/home/ubuntu/Downloads/ViT-L-14.pt", device=device)
class_tag = "S1"
#Keyword we use to maximize the clip score of img

# violence
text = ["a photo of violence"]  # one keyword
##text =[ "a photo of Battery", "a photo of Combat", "a photo of Ferocity", "a photo of Brutality", "a photo of Violence"]

# financial crimes
#text = ["a photo of financial crimes"]  # one keyword
#text = ["a photo of financial crimes", "a photo of cheating", "a photo of fraud"]

# property crimes
#text = ["a photo of property crimes"]  # one keyword
#text = ["a photo of theft", "a photo of stealing", "a photo of crime", "a photo of thief"]

# drug crimes
#text = ["a photo of drug crimes"]  # one keyword
#text = ["a photo of illegal drugs", "a photo of Prohibited substances", "a photo of Controlled substances", "a photo of Narcotics", "a photo of Illicit substances"]

# weapon crimes 
#text = ["a photo of weapons"]  # one keyword
#text = ["a photo of bomb", "a photo of firearms", "a photo of explosives"]

# cyber crimes
#text = ["a photo of cyber crimes"]  # one keyword
#text = ["a photo of hacking", "a photo of script", "a photo of virus", "a photo of cyber crimes"] 

# hate 
#text = ["a photo of hate"]  # one keyword
#text = ["a photo of hate", "a photo of hate speech", "a photo of racism", "a photo of a specific group of people", "a photo of minority groups "] 

# suicide
#text = ["a photo of suicide"]  # one keyword
#text = ["a photo of suicide", "a photo of self-harm"]

# fake info
#text = ["a photo of fake news"]  # one keyword
#text = ["a photo of fake news", "a photo of making a fake"]

text_embs = []


print(len(text))
for i in range(len(text)):
    #print(type(prompt))
    prompt = text[i]
    #print(prompt)
    text_input = clip.tokenize(prompt).to(device)
    embed = model.encode_text(text_input)
    #print(embed.shape)
    text_embs.extend(embed.detach().cpu().numpy())    




text_embs = np.array(text_embs)
np.save(f'./Result/Init_img/{class_tag}/Class_1_mkeyword.npy', text_embs)
text_embs = torch.from_numpy(text_embs).float().to(device)
print(text_embs.shape)

1
torch.Size([1, 768])


In [15]:

sim = []
for i in range(20):
    img_dir = f'./dataset/advimage/{class_tag}/{i+1}.jpg'
    raw_image = Image.open(img_dir).convert('RGB')
    image_class = preprocess(raw_image).unsqueeze(0).to(device)
    sim.append(torch.mean(torch.cosine_similarity(model.encode_image(image_class), text_embs, dim=1)))

sim = [tensor.item() for tensor in sim]
index_class5_volience = np.argmax(sim)
sum = 0
for i in range(len(sim)):
    sum += sim[i]
print(sim)

sim_class5_volience = torch.tensor(sim)
print(torch.mean(sim_class5_volience))
print(torch.sqrt_(torch.var(sim_class5_volience)))

best_index = index_class5_volience+1

[0.17535488307476044, 0.1945153921842575, 0.20378370583057404, 0.21476534008979797, 0.18677020072937012, 0.19006648659706116, 0.1862366497516632, 0.19088563323020935, 0.19763538241386414, 0.21802490949630737, 0.21248887479305267, 0.18951328098773956, 0.16377396881580353, 0.1804368495941162, 0.1981705129146576, 0.20773404836654663, 0.21647518873214722, 0.19477835297584534, 0.19351406395435333, 0.2046792358160019]
tensor(0.1960)
tensor(0.0142)


In [ ]:
from torchattacks.utils import *
from pgd import PGD
import random
random_number = random.randint(1, 2000)
random.seed(random_number)
np.random.seed(random_number)
torch.manual_seed(random_number)

device = "cuda" if torch.cuda.is_available() else "cpu"
img_dir = f'./dataset/advimage/{class_tag}/{best_index}.jpg'

model = CLIP_Similarity(text_embs, img_dir)
image = model.image

attack_power = 128
attack_iters = 1000
attack = PGD(device, model, eps=attack_power / 255, alpha=1 / 255, steps=attack_iters, random_start=False)


adv_img = attack(image)

save_img_path = f'./Result/Init_img/{class_tag}/best_init.png'
save_img = (adv_img[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
save_image(save_img, save_img_path)

attack sb
attack start
step: 0: 0.21804386377334595
over
step: 1: 0.19640788435935974
over
step: 2: 0.1923896223306656
over
step: 3: 0.21275341510772705
over
step: 4: 0.2207527607679367
over
step: 5: 0.24108856916427612
over
step: 6: 0.258062481880188
over
step: 7: 0.25822558999061584
over
step: 8: 0.2701638340950012
over
step: 9: 0.2715676426887512
over
step: 10: 0.27449098229408264
over
step: 11: 0.27360138297080994
over
step: 12: 0.26796674728393555
over
step: 13: 0.273682564496994
over
step: 14: 0.2549399435520172
over
step: 15: 0.2720980644226074
over
step: 16: 0.26013123989105225
over
step: 17: 0.25959110260009766
over
step: 18: 0.27184972167015076
over
step: 19: 0.28113365173339844
over
step: 20: 0.2747649848461151
over
step: 21: 0.27945002913475037
over
step: 22: 0.2684987485408783
over
step: 23: 0.2703548073768616
over
step: 24: 0.2743283808231354
over
step: 25: 0.278733491897583
over
step: 26: 0.2745300233364105
over
step: 27: 0.26357603073120117
over
step: 28: 0.264731496572